### Percobaan 1: Matching untuk 3 Driver dan Customer
#### Distance & Time(dgn Openrouteservice), Order Count & Rating.
*) Referensi code dari project semester lalu.

In [1]:
import pandas as pd
import pyomo.environ as pyo
import openrouteservice as ors
from dotenv import load_dotenv
import os

In [2]:
# Melakukan loading data client dan driver dari CSV.
# File dummy dari project semester lalu.

clients = pd.read_csv('../datasets/data_client.csv')
drivers = pd.read_csv('../datasets/data_driver.csv')

print(type(clients))

clients.head(3), drivers.head(3)

<class 'pandas.core.frame.DataFrame'>


(   id  latitude   longitude
 0   0 -7.251299  112.789335
 1   1 -7.247283  112.756581
 2   2 -7.344951  112.760688,
    id  latitude   longitude  rating  order_count  total_trip
 0   0 -7.293717  112.747264       1          180    1571.400
 1   1 -7.360642  112.716350       0           71     143.562
 2   2 -7.257948  112.731777       3           83     489.202)

In [3]:
# Load top 3 data pertama menjadi list

clients_data = clients.sample(n=3).to_dict('records')
drivers_data = drivers.sample(n=3).to_dict('records')
clients_data, drivers_data

([{'id': 838, 'latitude': -7.329288999994914, 'longitude': 112.70017199987907},
  {'id': 247, 'latitude': -7.275698999987424, 'longitude': 112.74634099976252},
  {'id': 1435,
   'latitude': -7.233198999981483,
   'longitude': 112.7275729998099}],
 [{'id': 679,
   'latitude': -7.298530999990615,
   'longitude': 112.72326599982075,
   'rating': 2,
   'order_count': 199,
   'total_trip': 1056.8889999999276},
  {'id': 857,
   'latitude': -7.355713999998608,
   'longitude': 112.70030099987876,
   'rating': 0,
   'order_count': 111,
   'total_trip': 321.677999999989},
  {'id': 1263,
   'latitude': -7.274541999987262,
   'longitude': 112.74919999975528,
   'rating': 2,
   'order_count': 171,
   'total_trip': 1531.1339999998688}])

In [4]:
memo = {}

In [5]:
# Definisikan weight.
weights = {
    'distance'      : { 'target': 0, 'positive': 1, 'negative': -1 },
    'duration'      : { 'target': 0, 'positive': 1, 'negative': -1 },
    'order_count'   : { 'target': 0, 'positive': 10, 'negative': -10 },
    'rating'        : { 'target': 0, 'positive': -1, 'negative': 1 },
}

# Simpan key dari ORS
load_dotenv(override=True)
key = os.getenv('OPENROUTESERVICE_KEY')
client = ors.Client(key=key, base_url='https://api.openrouteservice.org/')

In [6]:
# Buat mapping utk distance, duration, order count, dan rating dari masing-masing pasangan customer dan driver.

def get_distance_and_duration(lat1, lon1, lat2, lon2): 
    routes = client.directions(((lon1, lat1), (lon2, lat2)), profile='driving-car')
    return {
            "distance": routes.get('routes')[0]['summary']['distance'], 
            "duration": routes.get('routes')[0]['summary']['duration']
        }

data: dict[tuple, dict[str, any]] = {}
for d in drivers_data:
    d_id = d['id']
    data[d_id] = {}
    for c in clients_data:
        c_id = c['id']
        result = memo.get((d['latitude'], d['longitude'], c['latitude'], c['longitude']))
        if result is None:
            result = get_distance_and_duration(d['latitude'], d['longitude'], c['latitude'], c['longitude'])
            memo[(d['latitude'], d['longitude'], c['latitude'], c['longitude'])] = result

        data[d_id][c_id] = {
            "distance": result['distance'],
            "duration": result['duration'],
            
            "order_count": d['order_count'],
            "rating": d['rating'],
            # "rating_driver": d['rating_driver'],
            # "idle_time": d['idle_time'],

            # "rating_client": c['rating_client'],
        }

data

ConnectionError: HTTPSConnectionPool(host='api.openrouteservice.org', port=443): Max retries exceeded with url: //v2/directions/driving-car/json (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000000993CA165F0>: Failed to establish a new connection: [Errno 11002] getaddrinfo failed'))

In [ ]:
# Model pyomo utk solve GP
model = pyo.ConcreteModel()
model.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT)

# Define Variables
drivers = [d['id'] for d in drivers_data]
clients = [c['id'] for c in clients_data]

model.x = pyo.Var(drivers, clients, domain=pyo.NonNegativeIntegers)

# Define Objective
def get_penalty(target, val, positive=True):
  if positive:
    if val > target: return val - target
    else: return 0
  else:
    if val < target: return target - val
    else: return 0

# Penalty = (weight positive * X * deviasi dgn target / 1% dari target)
def penalty(c, d, param):
  m = weights[param]
  return (
    # (m['positive'] * model.x[d, c] * get_penalty(m['target'], data[d][c][param]) / m['target'] / 100)+ 
    # (m['negative'] * model.x[d, c] * get_penalty(m['target'], data[d][c][param], False) / m['target'] / 100)
    (m['positive'] * model.x[d, c] * get_penalty(m['target'], data[d][c][param], True)) + 
    (m['negative'] * model.x[d, c] * get_penalty(m['target'], data[d][c][param], False))
  )

def v_penalty(c, d, param):
  m = weights[param]
  return (
    (m['positive'] * 1 * get_penalty(m['target'], data[d][c][param], True)) + 
    (m['negative'] * 1 * get_penalty(m['target'], data[d][c][param], False))
  )

# Objective fn = Sum of penalty semua bidang.
model.Cost = pyo.Objective(
    expr = sum([
        penalty(c, d, 'distance') + 
        penalty(c, d, 'duration') + 
        penalty(c, d, 'order_count') + 
        penalty(c, d, 'rating') 
        for c in clients for d in drivers
      ]),
    sense = pyo.minimize)

# Define Constraint
# Constraint = jumlah dari setiap Driver dengan semua pasangan Clientnya adalah 1
model.driver = pyo.ConstraintList()
for d in drivers:
  model.driver.add(sum(model.x[d, c] for c in clients) == 1)

model.client = pyo.ConstraintList()
for c in clients:
  model.client.add(sum(model.x[d, c] for d in drivers) == 1)

In [ ]:
# Gunakan solver utk solve
result = pyo.SolverFactory('cbc').solve(model)
result.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 22882.6
  Upper bound: 22882.6
  Number of objectives: 1
  Number of constraints: 6
  Number of variables: 9
  Number of nonzeros: 9
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.01
  Wallclock time: 0.01
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created 

In [ ]:
print("Hasil dari Assignment Matching\n")
df = pd.DataFrame([], columns=['DRIVER ID',*clients])
df['DRIVER ID'] = drivers

for c in clients:
  mdr = []
  for d in drivers:
    mdr.append(model.x[d,c]())
  df[c] = mdr

df

Hasil dari Assignment Matching



,DRIVER ID,1225,429,524
0,1439,0.0,0.0,1.0
1,165,1.0,0.0,0.0
2,61,0.0,1.0,0.0


In [ ]:
from pprint import pprint

dic = df.to_dict('records')
n_dic = {}
for i in dic:
        driver_id = i.pop('DRIVER ID')
        n_dic[driver_id] = i

def find_pen(i):
        for j in clients:
                print(f"---- Penalties for Driver {i} -> Client {j}: ")
                distance_pen = v_penalty(j, i, 'distance')
                duration_pen = v_penalty(j, i, 'duration')
                rating_pen = v_penalty(j, i, 'rating')
                order_count_pen = v_penalty(j, i, 'order_count')
                print("\tDistance:", distance_pen)
                print("\tDuration:", duration_pen)
                print("\tRating:", rating_pen)
                print("\tOrder count:", order_count_pen)
                print("\tTotal penalty:", distance_pen + duration_pen + rating_pen + order_count_pen)

for i in drivers:
        print(f"Driver {i}:")
        find_pen(i)
        for j in clients:
                if n_dic[i][j] == 1:
                        print(f"Match: Driver {i} -> Client {j}")
                        
        pprint(data[i], underscore_numbers=True, compact=True)
        print()


Driver 1439:
---- Penalties for Driver 1439 -> Client 1225: 
	Distance: 15162.1
	Duration: 946.4
	Rating: -3
	Order count: 2000
	Total penalty: 18105.5
---- Penalties for Driver 1439 -> Client 429: 
	Distance: 8735.6
	Duration: 693.7
	Rating: -3
	Order count: 2000
	Total penalty: 11426.300000000001
---- Penalties for Driver 1439 -> Client 524: 
	Distance: 5447.5
	Duration: 455.2
	Rating: -3
	Order count: 2000
	Total penalty: 7899.7
Match: Driver 1439 -> Client 524
{429: {'distance': 8735.6, 'duration': 693.7, 'order_count': 200, 'rating': 3},
 524: {'distance': 5447.5, 'duration': 455.2, 'order_count': 200, 'rating': 3},
 1225: {'distance': 15162.1,
        'duration': 946.4,
        'order_count': 200,
        'rating': 3}}

Driver 165:
---- Penalties for Driver 165 -> Client 1225: 
	Distance: 2457.5
	Duration: 173.3
	Rating: 0
	Order count: 890
	Total penalty: 3520.8
---- Penalties for Driver 165 -> Client 429: 
	Distance: 14242.8
	Duration: 1100.7
	Rating: 0
	Order count: 890
	Total